## Analyze A/B Test Results

You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).  **Please save regularly.**

This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!

## Table of Contents
- [Introduction](#intro)
- [Part I - Probability](#probability)
- [Part II - A/B Test](#ab_test)
- [Part III - Regression](#regression)


<a id='intro'></a>
### Introduction

A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these 

For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.

**As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.** The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the [RUBRIC](https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric).

<a id='probability'></a>
#### Part I - Probability

To get started, let's import our libraries.

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)

`1.` Now, read in the `ab_data.csv` data. Store it in `df`.  **Use your dataframe to answer the questions in Quiz 1 of the classroom.**

a. Read in the dataset and take a look at the top few rows here:

In [ ]:
df = pd.read_csv('ab_data.csv')

In [ ]:
df.head()

b. Use the below cell to find the number of rows in the dataset.

In [ ]:
df.shape

c. The number of unique users in the dataset.

In [ ]:
df.user_id.nunique()

d. The proportion of users converted.

In [ ]:
df.converted.mean()

e. The number of times the `new_page` and `treatment` don't line up.

Finding inconsistencies in `df`, between the number of values in treatment that have not the value of `landing_page` as `new_page`.

#### `a.` Values for the treatment group.

In [ ]:
treatment = df['group'] == 'treatment'

df[treatment]['landing_page'].value_counts()

#### `b.` Values for the control group.

In [ ]:
control = df['group'] == 'control'

df[control]['landing_page'].value_counts()

#### `c.` We have a total of 3893 inconsistencies in our `df`.

In [ ]:
df[control]['landing_page'].value_counts()[1] + df[treatment]['landing_page'].value_counts()[1]

f. Do any of the rows have missing values?

In [ ]:
df.isnull().sum()

> **Answer:** No.

`2.` For the rows where **treatment** is not aligned with **new_page** or **control** is not aligned with **old_page**, we cannot be sure if this row truly received the new or old page.  Use **Quiz 2** in the classroom to provide how we should handle these rows.  

a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in **df2**.

In order to avoid inaccuracies in our analysis, we should drop the rows that have inconsistencies.

In [ ]:
new_page = df['landing_page'] == 'new_page'
old_page = df['landing_page'] == 'old_page'

In [ ]:
inconsistencies_labels = (df[treatment][old_page].index).union(df[control][new_page].index)

# check if the union worked properly - this should be 3893
inconsistencies_labels.shape[0]

In [ ]:
df_clean = df.drop(labels=inconsistencies_labels)

# check if all of the correct rows were removed - this should be 0
df_clean[((df_clean['group'] == 'treatment') == (df_clean['landing_page'] == 'new_page')) == False].shape[0]

`3.` Use **df2** and the cells below to answer questions for **Quiz3** in the classroom.

a. How many unique **user_id**s are in **df2**?

In [ ]:
df_clean.user_id.nunique()

b. There is one **user_id** repeated in **df2**.  What is it?

Do we have any duplicates? As other columns than `user_id` having duplicates wouldn't be a problem, I will only check duplicates for `user_id`.

In [ ]:
df_clean.duplicated('user_id').sum()

In [ ]:
duplicate_id = df_clean[df_clean.duplicated('user_id')]['user_id'].values[0]

In [ ]:
df_clean[df_clean['user_id'] == duplicate_id]

The difference between these columns is simply the timeframe, what suggests that the same user accessed the webpage more than one time. However, for the sake of this analysis, we shouldn't be counting the same `user_id` twice, reason why I will drop one of the rows.

In [ ]:
df_clean.drop(labels=[1899], inplace=True)

In [ ]:
df_clean.to_csv('ab_data_clean.csv', index=False)

## Analysis

Now that the dataframe is clean, we can begin our analysis. 

1. Let's start our analysis by checking some summary statistics.

In [ ]:
df = pd.read_csv('ab_data_clean.csv')

In [ ]:
df.head()

### Summary Statistics

`a.` **What is the probability of an individual converting regardless of the page they receive?**

In [ ]:
# since the conversion is represented by 1 in a [0, 1] column, we can get the proportion by calculating the mean
df['converted'].mean()

#### `b.` Given that an individual was in the `control` group, what is the probability they converted?

In [ ]:
control_group = df['group'] == 'control'

control_conversion = df[control_group]['converted']

control_mean = control_conversion.mean()

control_mean

#### `c.` Given that an individual was in the `treatment` group, what is the probability they converted?

In [ ]:
treatment_group = df['group'] == 'treatment'

treatment_conversion = df[treatment_group]['converted']

treatment_mean = treatment_conversion.mean()

treatment_mean

#### `d.` What is the probability that an individual received the new page?

In [ ]:
new_page_group = df['landing_page'] == 'new_page'

df[new_page_group].shape[0]/df.shape[0]

> **Conclusion**: We saw that the control group has a higher conversion than the treatment group (difference of ~ 0.158%), while the probability that an individual reveives the new page is practically the same as that an individual receives the old page. However, hypothesis tests are performed based on population, and there is no way we can infer that the statistics obtained with this sample are representative of the parameter. In order to fail to accept the Null Hypothesis ($H_0$), we must continue our investigation, by simulating a normal distribution based on the variance.

<a id='ab_test'></a>
### Part II - A/B Test

Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.  

However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?  

These questions are the difficult parts associated with A/B tests in general.  


`1.` For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of **$p_{old}$** and **$p_{new}$**, which are the converted rates for the old and new pages.

$$H_0: p_{new} - p_{old} \leq 0 $$
$$H_1: p_{new} - p_{old} > 0 $$

$$ \alpha: 0.05$$

`2.` Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the **converted** success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the **converted** rate in **ab_data.csv** regardless of the page. <br><br>

Use a sample size for each page equal to the ones in **ab_data.csv**.  <br><br>

Perform the sampling distribution for the difference in **converted** between the two pages over 10,000 iterations of calculating an estimate from the null.  <br><br>

Use the cells below to provide the necessary| parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use **Quiz 5** in the classroom to make sure you are on the right track.<br><br>

$p_{new}$ = $p_{old}$

In [ ]:
df.shape[0]

a. What is the **convert rate** for $p_{new}$ under the null? 

In [ ]:
p_new = df['converted'].mean()
p_new

b. What is the **convert rate** for $p_{old}$ under the null?

In [ ]:
p_old = df['converted'].mean()
p_old

c. What is $n_{new}$, the number of individuals?

In [ ]:
n_new = df[treatment_group].shape[0]
n_new

d. What is $n_{old}$?

In [ ]:
n_old = df[control_group].shape[0]
n_old

e. Simulate $n_{new}$ transactions with a convert rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in **new_page_converted**.

In [ ]:
new_page_converted = np.random.choice(2, n_new, p=[1 - p_new, p_new])

In [ ]:
new_page_converted.mean()

f. Simulate $n_{old}$ transactions with a convert rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in **old_page_converted**.

In [ ]:
old_page_converted = np.random.choice(2, n_old, p=[1 - p_old, p_old])

In [ ]:
old_page_converted.mean()

g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).

In [ ]:
old_page_converted.mean() - new_page_converted.mean()

h. Simulate 10,000 $p_{new}$ - $p_{old}$ values using this same process similarly to the one you calculated in parts **a. through g.** above.  Store all 10,000 values in a numpy array called **p_diffs**.

In [ ]:
p_diffs = []

for i in range(10000):
    mean_old = np.random.choice(2, n_old, p=[1 - p_old, p_old]).mean()
    mean_new = np.random.choice(2, n_new, p=[1 - p_new, p_new]).mean()
    mean_diff = mean_old - mean_new
    p_diffs.append(mean_diff)

p_diffs = np.array(p_diffs)

i. Plot a histogram of the **p_diffs**.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.

In [ ]:
sample_mean_diff = control_mean - treatment_mean

In [ ]:
plt.hist(p_diffs)

plt.axvline(sample_mean_diff, color='r');

j. What proportion of the **p_diffs** are greater than the actual difference observed in **ab_data.csv**?

In [ ]:
(p_diffs > sample_mean_diff).mean()

k. In words, explain what you just computed in part **j.**  What is this value called in scientific studies?  What does this value mean in terms of whether or not there is a difference between the new and old pages?

>In part `j.` I computed the **p-value**, which measures the probability of finding the statistics for the alternative hypothesis, or to the extreme in favor of the alternative hypothesis (in this case greater than the minimum possible statistics), considering that the null hypothesis is true. This means that there is a greater chance than we are willing to admit ($\alpha$) that the mean parameter of the population ($\mu_{p}$) falls within the distribution of the null, confirming the null hypothesis.

l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let `n_old` and `n_new` refer the the number of rows associated with the old page and new pages, respectively.

In [ ]:
df[df['group'] == 'treatment'].shape[0]

In [ ]:
import statsmodels.api as sm

convert_old = df[df['group'] == 'control'].converted.sum()
convert_new = df[df['group'] == 'treatment'].converted.sum()
n_old = df[df['group'] == 'control'].shape[0]
n_new = df[df['group'] == 'treatment'].shape[0]

m. Now use `stats.proportions_ztest` to compute your test statistic and p-value.  [Here](http://knowledgetack.com/python/statsmodels/proportions_ztest/) is a helpful link on using the built in.

In [ ]:
from statsmodels.stats import proportion

In [ ]:
zcore, pval = proportion.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='larger')

zcore, pval

n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts **j.** and **k.**?

> The z-score of value ~ 1.31 means that the value found for sample difference is roughly 1.31 standard deviations above the mean ($1.31\sigma.\bar{x}$), which confirms the findings of parts `j.` and `k.`,  as computed below through `zcore * p_diffs.std()`, which should equal the `sample_mean_diff`. 

In [ ]:
zcore * p_diffs.std()

In [ ]:
sample_mean_diff

<a id='regression'></a>
### Part III - A regression approach

`1.` In this final part, you will see that the result you acheived in the previous A/B test can also be acheived by performing regression.<br><br>

a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?

> **Answer**: We should be performing a Logistic Regression, given that Logistic Regression is a technique used to predict two possible outcomes.

b. The goal is to use **statsmodels** to fit the regression model you specified in part **a.** to see if there is a significant difference in conversion based on which page a customer receives.  However, you first need to create a column for the intercept, and create a dummy variable column for which page each user received.  Add an **intercept** column, as well as an **ab_page** column, which is 1 when an individual receives the **treatment** and 0 if **control**.

In [ ]:
df['intercept'] = 1

df = df.join(pd.get_dummies(df['group']))

In [ ]:
df['ab_page'] = df['treatment']
df.drop(columns=['control', 'treatment'], inplace=True)

c. Use **statsmodels** to import your regression model.  Instantiate the model, and fit the model using the two columns you created in part **b.** to predict whether or not an individual converts.

In [ ]:
mod = sm.Logit(df['converted'], df[['intercept', 'ab_page']])

d. Provide the summary of your model below, and use it as necessary to answer the following questions.

In [ ]:
results = mod.fit()
results.summary()

e. What is the p-value associated with **ab_page**? Why does it differ from the value you found in **Part II**?

> **Answer**: The p-value associated with `ab_page` is 0.190. This happens because the Null Hypothesis and Alternative Hypothesis in the Logistic Regression refer to the probability of existing a relationship between the `ab_page` vaiable and the `converted` variable.

f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?

> **Answer**: The advantages of adding variables into the regression model is it may increase the model's precision, increasing its predictions reliablitiy. On the other hand, adding too many variables to a regression model may cause overfitting, which causes the model to work best for the available data, but fail to predict the future.

g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives. You will need to read in the **countries.csv** dataset and merge together your datasets on the approporiate rows.  [Here](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html) are the docs for joining tables. 

Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - **Hint: You will need two columns for the three dummy variables.** Provide the statistical output as well as a written response to answer this question.

In [ ]:
countries_df = pd.read_csv('./countries.csv')
df = countries_df.set_index('user_id').join(df.set_index('user_id'), how='inner')

In [ ]:
df = df.join(pd.get_dummies(df_new['country']))

In [ ]:
mod2 = sm.Logit(df['converted'], df[['intercept', 'UK', 'CA']])
results2 = mod2.fit()
results2.summary()

In [ ]:
np.exp(results2.params)

> **Conclusions**: The country does not seem to be relevant to predict the conversion, considering the P-values of each considered variable above. The countries also do not have difference amongst each other, considering all other variables constant. For each country, it is almost the same as likely as other countries (by factor of ~ 1.0) that the new page will convert at a fixed ratio when compared to the old page.

h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.  

Provide the summary results, and your conclusions based on the results.

In [ ]:
df['ab_page_squared'] = df['ab_page']*df['ab_page']

In [ ]:
mod3 = sm.Logit(df['converted'], df[['intercept', 'ab_page', 'CA', 'UK']])
results3 = mod3.fit()
results3.summary()

In [ ]:
np.exp(results3.params)

> **Conclusions**: The analyzed variables failed to fit a logistic model that could predict the a future value for the conversion of the new website. In other words, the new site does not show a relevant change in the number of conversions that would justify its change.

<a id='conclusions'></a>
## Conclusions

Congratulations on completing the project! 

### Gather Submission Materials

Once you are satisfied with the status of your Notebook, you should save it in a format that will make it easy for others to read. You can use the __File -> Download as -> HTML (.html)__ menu to save your notebook as an .html file. If you are working locally and get an error about "No module name", then open a terminal and try installing the missing module using `pip install <module_name>` (don't include the "<" or ">" or any words following a period in the module name).

You will submit both your original Notebook and an HTML or PDF copy of the Notebook for review. There is no need for you to include any data files with your submission. If you made reference to other websites, books, and other resources to help you in solving tasks in the project, make sure that you document them. It is recommended that you either add a "Resources" section in a Markdown cell at the end of the Notebook report, or you can include a `readme.txt` file documenting your sources.

### Submit the Project

When you're ready, click on the "Submit Project" button to go to the project submission page. You can submit your files as a .zip archive or you can link to a GitHub repository containing your project files. If you go with GitHub, note that your submission will be a snapshot of the linked repository at time of submission. It is recommended that you keep each project in a separate repository to avoid any potential confusion: if a reviewer gets multiple folders representing multiple projects, there might be confusion regarding what project is to be evaluated.

It can take us up to a week to grade the project, but in most cases it is much faster. You will get an email once your submission has been reviewed. If you are having any problems submitting your project or wish to check on the status of your submission, please email us at dataanalyst-project@udacity.com. In the meantime, you should feel free to continue on with your learning journey by beginning the next module in the program.